# Decision Tree (Árvore de Decisão)

Determinar se após um telefonema de um banco de Portugual, o cliente fará um depósito na conta baseado em seu histórico pessoal, hitórico no banco e  tentativa de contato anteriores. Neste exemplo iremos utilizar o GridSearchCV para determinar os melhores parâmetros do modelo e melhorar a acurácia na amostra de teste

Dataset obtido no Kaggle (https://www.kaggle.com/volodymyrgavrysh/bank-marketing-campaigns-dataset)

Importanto bibliotecas usuais

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [31]:
dados = pd.read_csv('bank-additional-full.csv')

In [32]:
dados.head()

,"age;""job"";""marital"";""education"";""default"";""housing"";""loan"";""contact"";""month"";""day_of_week"";""duration"";""campaign"";""pdays"";""previous"";""poutcome"";""emp.var.rate"";""cons.price.idx"";""cons.conf.idx"";""euribor3m"";""nr.employed"";""y"""
0,"56;""housemaid"";""married"";""basic.4y"";""no"";""no"";..."
1,"57;""services"";""married"";""high.school"";""unknown..."
2,"37;""services"";""married"";""high.school"";""no"";""ye..."
3,"40;""admin."";""married"";""basic.6y"";""no"";""no"";""no..."
4,"56;""services"";""married"";""high.school"";""no"";""no..."


In [33]:
dados = pd.read_csv('bank-additional-full.csv',sep=';')

In [34]:
dados.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


age -> idade (variável numérica)

job -> emprego (variável categórica) 

marital -> estado civíl (variável categórica)

education -> nível de educação (variável categórica)

default -> já tem crédito? (variável categórica)

housing -> tem hipoteca? (variável categórica)

loan -> empréstimo pessoal? (variável categórica)

contact -> tipo de contato (variável categórica)

month -> mês do último contato (variável categórica)

day_of_week -> dia da semana da tentativa de contato (variável categórica)

duration -> duração da última chamada (variável numérica)

campaign -> número de ligações para o cliente (variável numérica)

pdays -> intervalo entre as duas últimas ligações (variável numérica)
 
previous -> número de ligações antes desta campanha (variável numérica)

poutcome -> resultado da última campanha (variável categórica)

emp.var.rate -> taxa da variaçào de emprego (variável numérica)

cons.price.idx -> indice de preço do consumidor (variável numérica)

cons.conf.idx -> indice de confiança do consumidor (variável numérica)

euribor3m -> indicador diário (variável numérica)

nr.employed -> número de funcionários (variável numérica)

y -> resultado (variável categórica)

In [35]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

Devido o número de ligações durante a semana estar balanceado, podemos remover esta informação

In [36]:
dados = dados.drop('day_of_week',axis=1)

Convertendo variáveis catgóricas em variáveis numéricas

Convertendo utilizando dicionário

In [37]:
dados['y'] = dados['y'].map({'yes' : 1, 'no' : 0})

In [38]:
dados.head()

,age,job,marital,education,default,housing,loan,contact,month,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
1,57,services,married,high.school,unknown,no,no,telephone,may,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
2,37,services,married,high.school,no,yes,no,telephone,may,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
3,40,admin.,married,basic.6y,no,no,no,telephone,may,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
4,56,services,married,high.school,no,no,yes,telephone,may,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0


Convertendo utilizando o LabelEnconder

In [39]:
from sklearn.preprocessing import LabelEncoder

In [40]:
enconder = LabelEncoder()

In [41]:
dados['job'] = enconder.fit_transform(dados['job'])
dados['marital'] = enconder.fit_transform(dados['marital'])
dados['education'] = enconder.fit_transform(dados['education'])
dados['default'] = enconder.fit_transform(dados['default'])
dados['housing'] = enconder.fit_transform(dados['housing'])
dados['loan'] = enconder.fit_transform(dados['loan'])
dados['contact'] = enconder.fit_transform(dados['contact'])
dados['month'] = enconder.fit_transform(dados['month'])
dados['poutcome'] = enconder.fit_transform(dados['poutcome'])

In [42]:
dados.head()

,age,job,marital,education,default,housing,loan,contact,month,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,3,1,0,0,0,0,1,6,261,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
1,57,7,1,3,1,0,0,1,6,149,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
2,37,7,1,3,0,2,0,1,6,226,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
3,40,0,1,1,0,0,0,1,6,151,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
4,56,7,1,3,0,0,2,1,6,307,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0


Algumas colunas numéricas os dados apresentam uma variação muito grande. Isto pode afetar a eficiência do modelo e por esta razão serão normalizados

In [43]:
colunas = ['age','duration','pdays','cons.price.idx','cons.conf.idx','euribor3m','nr.employed']

In [44]:
from sklearn.preprocessing import StandardScaler

In [45]:
sc=StandardScaler()

In [46]:
dados[colunas] = sc.fit_transform(dados[colunas])

In [47]:
dados.head()

,age,job,marital,education,default,housing,loan,contact,month,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,1.533034,3,1,0,0,0,0,1,6,0.010471,1,0.195414,0,1,1.1,0.722722,0.886447,0.71246,0.33168,0
1,1.628993,7,1,3,1,0,0,1,6,-0.421501,1,0.195414,0,1,1.1,0.722722,0.886447,0.71246,0.33168,0
2,-0.290186,7,1,3,0,2,0,1,6,-0.124520,1,0.195414,0,1,1.1,0.722722,0.886447,0.71246,0.33168,0
3,-0.002309,0,1,1,0,0,0,1,6,-0.413787,1,0.195414,0,1,1.1,0.722722,0.886447,0.71246,0.33168,0
4,1.533034,7,1,3,0,0,2,1,6,0.187888,1,0.195414,0,1,1.1,0.722722,0.886447,0.71246,0.33168,0


Separando as variaveis em X e Y

In [48]:
X = dados.drop('y',axis=1).values
Y = dados['y'].values

Separando em amostras de treino e teste

In [49]:
from sklearn.model_selection import train_test_split

In [50]:
X_treino,X_teste,Y_treino,Y_teste=train_test_split(X,Y,test_size=0.25,random_state=0)

Aplicando o modelo da arvore de decisão

In [51]:
from sklearn.tree import DecisionTreeClassifier

In [52]:
dte = DecisionTreeClassifier()

In [53]:
dte

DecisionTreeClassifier()

In [54]:
dte.get_params

<bound method BaseEstimator.get_params of DecisionTreeClassifier()>

Realizando ajuste do modelo

In [55]:
dados.dtypes

age               float64
job                 int32
marital             int32
education           int32
default             int32
housing             int32
loan                int32
contact             int32
month               int32
duration          float64
campaign            int64
pdays             float64
previous            int64
poutcome            int32
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
y                   int64
dtype: object

In [56]:
dte.fit(X_treino,Y_treino)

DecisionTreeClassifier()

Determinando acurácia na amostra de treino

In [57]:
score_treino = dte.score(X_treino,Y_treino)

In [58]:
score_treino

1.0

Realizando previsão na amostra de teste

In [59]:
Y_previsto = dte.predict(X_teste)

Gerando matriz de confusão para comparar os resultados

In [60]:
from sklearn.metrics import confusion_matrix

In [61]:
cm=confusion_matrix(Y_teste,Y_previsto)

In [62]:
cm

array([[8569,  570],
       [ 538,  620]], dtype=int64)

In [63]:
score_teste = dte.score(X_teste,Y_teste)

In [64]:
score_teste

0.8923958434495484

Apesar do score na amostra de teste ser elevado, devemos notar que muitos casos positivos foram classificados como negativo. Isto pode ser corrigido, ajustando melhor os parâmetros do modelo com GridSearch por exeplo, ou balanceando a amostra

Criando modelo do GridSearchCV

In [65]:
criterion=['gini','entropy']
max_depth = np.arange(2,20)
max_features= np.arange(2,20)

In [66]:
from sklearn.model_selection import GridSearchCV

In [67]:
parametros = {'criterion' : criterion, 'max_depth' : max_depth, 'max_features' : max_features}

In [68]:
melhor_modelo = GridSearchCV(dte, parametros, n_jobs=-1, cv=5, refit=True, scoring='accuracy')

In [69]:
melhor_modelo.fit(X_treino, Y_treino)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19]),
                         'max_features': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19])},
             scoring='accuracy')

In [39]:
modelo_final = melhor_modelo.best_estimator_

In [40]:
modelo_final

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=17, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [41]:
Y_previsto = modelo_final.predict(X_teste)

In [42]:
cm_nova=confusion_matrix(Y_teste,Y_previsto)

In [43]:
cm_nova

array([[8824,  315],
       [ 524,  634]])

In [44]:
score_teste = modelo_final.score(X_teste,Y_teste)

In [45]:
score_teste

0.9185199572691075